In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# Concatenate Train and Test Data (For Applying All Processes In Same Time)

In [ ]:
target = train_data['SalePrice']
test_ids = test_data['Id']

train1 = train_data.drop(['SalePrice', 'Id'], axis=1)
test1 = test_data.drop(['Id'], axis=1)

data = pd.concat([train1, test1], axis=0).reset_index(drop=True)
data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2915,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2916,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
2917,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,NaN,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,

# Cleaning

In [ ]:
data1 = data.copy()

In [ ]:
data1['MSSubClass'] = data1['MSSubClass'].astype(str) #MSSubClass is categorical data

## Categorical Missing Values

In [ ]:
for col in [
    'Alley',
    'BsmtQual',
    'BsmtCond',
    'BsmtExposure',
    'BsmtFinType1',
    'BsmtFinType2',
    'FireplaceQu',
    'GarageType',
    'GarageFinish',
    'GarageQual',
    'GarageCond',
    'PoolQC',
    'Fence',
    'MiscFeature'
]:
  data1[col] = data1[col].fillna('None')

for col in [
    'MSZoning',
    'Utilities',
    'Exterior1st',
    'Exterior2nd',
    'MasVnrType',
    'Electrical',
    'KitchenQual',
    'Functional',
    'SaleType'
]:
  data1[col] = data1[col].fillna(data1[col].mode()[0])

In [ ]:
data2 = data1.copy()

## Numeric Missing Values

In [ ]:
from sklearn.impute import KNNImputer

numerical_cols = [col for col in data2.columns
                  if data2[col].dtype in ['int64', 'float64']]

imputer = KNNImputer()
imputed_cols = pd.DataFrame(imputer.fit_transform(data2[numerical_cols]))
imputed_cols.columns = imputer.get_feature_names_out()
data3 = pd.concat([imputed_cols, data2.drop(numerical_cols, axis=1)], axis=1)

In [ ]:
data4 = data3.copy()

# Data Transformation

## Log Transformation

In [ ]:
from scipy.stats import skew

skewness = data4[numerical_cols].apply(lambda x: skew(x))

for col in skewness.index:
  if(abs(skewness[col] >= 0.5)):
    data4[col] = np.log1p(data4[col])

In [ ]:
log_target = np.log(target)

In [ ]:
data5 = data4.copy()

# Encoding For Categorical Values

In [ ]:
data6 = pd.get_dummies(data5, dtype='float')

#Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data7 = pd.DataFrame(scaler.fit_transform(data6), index=data6.index, columns=data6.columns)

# Split Data

In [ ]:
train = data7.loc[:train_data.index.max(), :].copy()
test = data7.loc[train_data.index.max() + 1:, :].reset_index(drop=True).copy()

# Model Selection

In [ ]:
from pycaret.regression import setup, compare_models

setup(data=pd.concat([train, log_target], axis=1), target='SalePrice', preprocess=False)

,Description,Value
0,Session id,7297
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1460, 317)"
4,Transformed data shape,"(1460, 317)"
5,Transformed train set shape,"(1021, 317)"
6,Transformed test set shape,"(439, 317)"
7,Numeric features,316


In [ ]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.0844,0.0161,0.1254,0.8940,0.0097,0.0071,6.7030
gbr,Gradient Boosting Regressor,0.0915,0.0177,0.1316,0.8822,0.0102,0.0077,0.6060
omp,Orthogonal Matching Pursuit,0.0900,0.0196,0.1360,0.8738,0.0104,0.0075,0.0520
br,Bayesian Ridge,0.0884,0.0199,0.1370,0.8709,0.0105,0.0074,0.1390
lightgbm,Light Gradient Boosting Machine,0.0949,0.0197,0.1389,0.8700,0.0108,0.0079,0.6940
rf,Random Forest Regressor,0.1015,0.0224,0.1482,0.8542,0.0115,0.0085,2.3630
xgboost,Extreme Gradient Boosting,0.1029,0.0222,0.1473,0.8528,0.0114,0.0086,0.8210
et,Extra Trees Regressor,0.1031,0.0227,0.1495,0.8521,0.0116,0.0086,2.3050
ridge,Ridge Regression,0.0932,0.0229,0.1459,0.8506,0.0112,0.0078,0.1030
ada,AdaBoost Regressor,0.1340,0.0321,0.1783,0.7909,0.0138,0.0112,0.5380


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [ ]:
from sklearn.linear_model import BayesianRidge
model = BayesianRidge(verbose=1)

model.fit(train, log_target)

Convergence after  6  iterations


BayesianRidge(verbose=1)

In [ ]:
from sklearn.model_selection import cross_val_score

results = cross_val_score(model, train, log_target, scoring='neg_mean_squared_error', cv=5)

Convergence after  6  iterations
Convergence after  6  iterations
Convergence after  6  iterations
Convergence after  6  iterations
Convergence after  5  iterations


In [ ]:
preds = model.predict(test)
preds = np.exp(preds)

submission = pd.DataFrame({'Id' : test_ids,
              'SalePrice' : preds})

submission.to_csv('submission.csv', index=False)

# Ensemble

In [ ]:
#!pip install catboost

In [ ]:
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [ ]:
models = {
    'catboost' : CatBoostRegressor(verbose=0),
    'br' : BayesianRidge(),
    'lgbm' : LGBMRegressor(verbose=0),
    'ridge' : Ridge(),
    'omp' : OrthogonalMatchingPursuit()
}

In [ ]:
param_grids = {
    'catboost': {
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'iterations': [100, 500, 1000]
    },
    'br': {
        'n_iter': [300, 500, 1000]
    },
    'lgbm': {
        'num_leaves': [31, 50, 100],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 500]
    },
    'ridge': {
        'alpha': [0.01, 0.1, 1, 10],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']
    },
    'omp': {
        'n_nonzero_coefs': [None, 100, 500],
        'fit_intercept': [True, False]
    }
}

In [ ]:
from sklearn.model_selection import GridSearchCV

best_models = {}

for name, model in models.items():
  grid_search = GridSearchCV(estimator=model,
                             param_grid=param_grids[name],
                             cv=5, n_jobs=-1, verbose=2)
  grid_search.fit(train, log_target)
  best_models[name] = grid_search.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [ ]:
results = {}

for name, model in best_models.items():
  result = np.exp(np.sqrt(-cross_val_score(model, train, log_target, scoring = 'neg_mean_squared_error', cv=10)))
  results[name] = result


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


In [ ]:
for name, result in results.items():
  print('-------\n' + name + '\n---------')
  print(np.mean(result))


-------
catboost
---------
1.1307257006787474
-------
br
---------
1.1399469659372081
-------
lgbm
---------
1.1364567850688045
-------
ridge
---------
1.1447888537267972
-------
omp
---------
1.1414391908328925


In [ ]:
final_results = (
    0.4 * np.exp(best_models['catboost'].predict(test))+
    0.2 * np.exp(best_models['br'].predict(test))+
    0.2 * np.exp(best_models['lgbm'].predict(test))+
    0.1 * np.exp(best_models['ridge'].predict(test))+
    0.1 * np.exp(best_models['omp'].predict(test))
)

In [ ]:
submission = pd.DataFrame({'Id' : test_ids,
              'SalePrice' : final_results})

submission.to_csv('submission2.csv', index=False)
submission

,Id,SalePrice
0,1461,122806.286827
1,1462,160285.155926
2,1463,184814.840695
3,1464,191106.487939
4,1465,190583.853926
...,...,...
1454,2915,82372.561014
1455,2916,80204.827638
1456,2917,166813.374470
1457,2918,114310.605212
